## This page is under development...

I'm using yfinance package to download the stock data from yahoo finance. Then I will try predicting the prices for each stock. My goal is to make a prediction on the expected value and variance of the price based on the previous values. Then I'll try to compute the experimental covarience between different stocks and input them to an optimization problem designed to reduce the mixed variance while maximizing the expected output. This is based on the mean-volatility predicate in financial analysis.  
My initial intention was to use the rather new *tensorflow probability* capability to estimate the mean a variance for each output point. After some trials and failings, I have decided to use a simpler approach:
 - Use a small Nueral Net to compute a predicted value for the next days opening price for each stock.
 - Assume that my predictor assumes a normal distribution around the observed values (I should check this!). Therefore I can use the unbiased estimator of SD to find the expected variance around my predicted price.
 - Compute the correlations between different stocks experimentally (i.e. pairwise correlation between data columns)
 - perform the optimization as before.

In [1]:
import importlib
import single_stock_predictor
importlib.reload(single_stock_predictor)
import pickle
import yfinance as yf
import pandas as pd
import tensorflow as tf
import pickle
import numpy as np
#import mdn
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import h5py
from scipy.optimize import minimize

%matplotlib notebook

If running for first time, you need to downloaded the stock symbols or "tickers". This is in done by setting parameter  "get_tickers".<br> Currently I'm downloading the daily data for 9years. If you already have downloaded some part of the data you can download the rest and append them to each other. Later I hope to automatize this section, since my goal is to run this script once weekly or so.

In [44]:
get_tickers = False 
read_tickers = False
get_histories = False
get_updated_data = False
read_data = False
read_updated_data = True
get_business_info = True #you can go through business info field to exlude companies you don't want to invest in
if get_updated_data or get_business_info:
    read_tickers = True

In [45]:
def download_tickers():
    !curl -o /Users/abnousa/software/smartop/nasdaqtraded_companylist.txt ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt
    symbols = pd.read_csv("/Users/abnousa/software/smartop/nasdaqtraded_companylist.txt", sep = "|")
    symbols = symbols.iloc[0:(symbols.shape[0] - 1),:] #last row is time
    tickers = {}
    failed = []
    for i in symbols.index:
        sym = symbols.iloc[i]['Symbol']
        ticker = yf.Ticker(sym)
        try:
            check = ticker.calendar
        except Exception as e:
            print(' '.join(["disregarding", sym, type(e).__name__]))
            failed.append(sym)
            continue
        print(' '.join([sym, 'added']))
        name = symbols.iloc[i]['Security Name']
        tickers[sym] = {'name': name, 'ticker': ticker}
    sym_data = {'tickers':tickers, 'failed':failed}
    with open('sym_data.pkl', 'wb') as symfile:
        pickle.dump(sym_data, symfile)
    return(sym_data)
if get_tickers:
    sym_data = download_tickers()

In [48]:
if read_tickers:
    with open('sym_data.pkl', 'rb') as symfile:
        sym_data = pickle.load(symfile)
        tickers, failed = sym_data['tickers'], sym_data['failed']

In [47]:
def download_business_info(tickers):
    for ticker in tickers.keys():
        #print(ticker)
        tickers[ticker]['business_summary'] = tickers[ticker]['ticker'].info.get('longBusinessSummary', None)
    with open('sym_data.pkl', 'wb') as symfile:
        pickle.dump(sym_data, symfile)
if get_business_info:
    download_business_info(tickers)

In [12]:
def download_histories(tickers):
    period = "9y"
    d = download_ticker_histories(tickers, period = period, interval = "1d", columns = ['Open'])
    with open("daily_history_9y.pkl", 'wb') as histfile:
        pickle.dump(d, histfile)
    return d
if get_histories:
    d = download_histories(sym_data['tickers'])

In [13]:
def download_ticker_histories(tickers, start = None, period = None, end = None, interval = "1d", columns = ['Open']):
    msft = yf.Ticker("MSFT")
    if start is None:
        temp_hist = msft.history(period="9y", interval="1d")
        end = list(temp_hist.index)[-1]
        start = list(temp_hist.index)[0]
    else:
        temp_hist = msft.history(start = start, interval = "1d")
        end = list(temp_hist.index)[-1]
    d = pd.DataFrame(data = 0, columns = list(tickers.keys()), index = temp_hist.index)
    for counter, i in enumerate(d.columns):
        if counter % 100 == 0:
            print(i)
        hist = (tickers[i]['ticker'].history(start = start, end = end, interval = interval)[columns]).drop_duplicates(keep = 'last')
        d[i] = hist
    return d

In [14]:
if read_data:
    with open("daily_history_9y.pkl", 'rb') as infile:
        d = pickle.load(infile)

In [24]:
def update_data(d, tickers, columns = ['Open']):
    last_date = (pd.to_datetime(d.index.values[-1])).strftime("%Y-%m-%d")
    d_update = download_ticker_histories(tickers, start = last_date, interval = "1d", columns = columns)
    d_update = d_update.iloc[1:,:]
    d_update.head()
    d_merged = pd.concat([d, d_update], axis = 0)
    with open("daily_history_updated.pkl", 'wb') as histfile:
        pickle.dump(d_merged, histfile)
    return d_merged
if get_updated_data:
    d = update_data(d, tickers, columns = ['Open'])

In [25]:
if read_updated_data:
    with open("daily_history_updated.pkl", 'rb') as infile:
        d = pickle.load(infile)

In [ ]:
plt.hist(d.isna().sum(), bins = 100)

In [ ]:
d = d.iloc[:, list(np.where(d.isna().sum() < 20)[0])]
d = d.fillna(method = "bfill")
d.shape

In [ ]:
import single_stock_predictor
importlib.reload(single_stock_predictor)

In [ ]:
stock_name = "MSFT"
weekly = False
window_size = 30
batch_size = 32
shuffle_buffer = None
distributional = False
epochs = 40
training_points = 1500
sd_estimate_required = True
model_outdir = "models"
training_verbosity = 1
look_ahead_window = 5
pred, train_sd, train_forecasts, valid_sd, valid_forecasts = single_stock_predictor.predict_tomorrow(stock_name, d, model_outdir = model_outdir, weekly = weekly, training_points = training_points, window_size = window_size, batch_size = batch_size, distributional = distributional, epochs = epochs, sd_estimate_required = sd_estimate_required, shuffle_buffer = shuffle_buffer, training_verbosity = training_verbosity, look_ahead_window = look_ahead_window)

In [ ]:
plt.figure()
single_stock_predictor.plot_predictions(d[stock_name], train_forecasts, train_sd, window_size, limit_begin = 0, limit_end = 200, look_ahead_window = look_ahead_window)

In [ ]:
plt.figure()
single_stock_predictor.plot_predictions(d[stock_name], train_forecasts, valid_sd, window_size, limit_begin = 0, limit_end = 200, look_ahead_window = look_ahead_window)

In [ ]:
plt.figure()
single_stock_predictor.plot_predictions(d[stock_name][training_points:], valid_forecasts, train_sd, window_size, limit_begin = 0, limit_end = 200, look_ahead_window = look_ahead_window)

In [ ]:
plt.figure()
single_stock_predictor.plot_predictions(d[stock_name][training_points:], valid_forecasts, valid_sd, window_size, limit_begin = 0, limit_end = 200, look_ahead_window = look_ahead_window)

In [ ]:
print(train_sd, valid_sd)

okay, so far we have showed that for the specified stock (here, MSFT aka microsoft) our model generates rather dependable predictions of price and our estimated standard deviation seems to be fitting at least visually. Of course, one can argue that we have picked an easy ticker, you'd expect microsoft to have a stable price. Well, I can't argue against that. But now, I'm going to randomly pick 10 tickers and perform the same operation on each of them. Before that I
m going to use the *cov()* function from pandas to compute pairwise correlation between the selected tickers.

In [ ]:
import random
ticker_set = random.sample(list(d.columns), 10)
ticker_set

In [ ]:
d_select = d[ticker_set]
print(d_select.shape)
corels = d_select.corr()

In [ ]:
covariances = d_select.cov()
covariances.head()


In [ ]:
corels.head()

In [ ]:
corels_matrix = np.array(corels)
heatmap(corels_matrix, corels.columns.values, corels.columns.values, cbarlabel = "correlation")

In [ ]:
weekly = False
window_size = 30
batch_size = 32
shuffle_buffer = None
distributional = False
epochs = 20
training_points = 1500
sd_estimate_required = True
model_outdir = "models"
training_verbosity = 0

In [ ]:
prediction_dict = {}
for stock_name in ticker_set:
    print(" ".join(["processing", stock_name]))
    pred, train_sd, train_forecasts, valid_sd, valid_forecasts = single_stock_predictor.predict_tomorrow(stock_name, d, model_outdir = model_outdir, weekly = weekly, training_points = training_points, window_size = window_size, batch_size = batch_size, distributional = distributional, epochs = epochs, sd_estimate_required = sd_estimate_required, shuffle_buffer = shuffle_buffer, training_verbosity = training_verbosity, look_ahead_window = look_ahead_window)
    prediction_dict[stock_name] = {'expected_tomorrow': pred,
                                   'train_sd': train_sd,
                                   'train_forecasts': train_forecasts,
                                   'valid_sd': valid_sd,
                                   'valid_forecasts': valid_forecasts}
    current_price = d.iloc[d.shape[0]-1][stock_name]
    return_rate = pred / current_price - 1
    prediction_dict[stock_name]['return_rate'] = return_rate

It looks like for a few of the stocks, validation error was smaller than the training error. I'm curious why.

So here are the plots from validation data:

In [ ]:
plt.figure()
plt.subplot(5,2,1)
for plt_index, stock_name in enumerate(ticker_set):
    ax = plt.subplot(5,2,plt_index + 1)
    single_stock_predictor.plot_predictions(d[stock_name][training_points:], prediction_dict[stock_name]['valid_forecasts'], prediction_dict[stock_name]['valid_sd'], window_size, limit_begin = 0, limit_end = 200, ax = ax, legend = False, look_ahead_window = look_ahead_window)
    
            

and here are plots for training data:

In [ ]:
plt.figure()
plt.subplot(5,2,1)
for plt_index, stock_name in enumerate(ticker_set):
    ax = plt.subplot(5,2,plt_index + 1)
    single_stock_predictor.plot_predictions(d[stock_name], prediction_dict[stock_name]['train_forecasts'], prediction_dict[stock_name]['valid_sd'], window_size, limit_begin = 0, limit_end = 200, ax = ax, legend = False, look_ahead_window = look_ahead_window)
    

In [ ]:
d_select.head()

In [ ]:
for stock_name in ticker_set:
    pred = prediction_dict[stock_name]['expected_tomorrow']
    current_price = d.iloc[d.shape[0] - 1, :][stock_name]
    return_rate = pred / current_price - 1
    prediction_dict[stock_name]['return_rate'] = return_rate

In [ ]:
rat_risks = pd.DataFrame({'return_rate':[prediction_dict[stock_name]['return_rate'] for stock_name in ticker_set],
                        'risk' : [prediction_dict[stock_name]['valid_sd'] for stock_name in ticker_set],
                         'price': d.iloc[d.shape[0] - 1][ticker_set]})

In what follows I'll be minimzing the mixed variance by selecting the "best" portfolio for a given return rate and budjet. For the optimization part I have heavily been dependent on the codes in [this tutorial](https://towardsdatascience.com/efficient-frontier-optimize-portfolio-with-scipy-57456428323e). Kudos to **J Li**. I have modified the code to compute the portfolio return rate and risk from mixture of normals distribution. Later I will also modify the optimization function by adding new constraints to compute the number of shares to buy with a given budget rather than the weight of instruments in the portfolio.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(rat_risks['risk'], rat_risks['return_rate'])
for stock_name in ticker_set:
    ax.annotate(stock_name, (prediction_dict[stock_name]['valid_sd'], prediction_dict[stock_name]['return_rate']))
plt.xlabel("risk")
plt.ylabel("return rate")
plt.show()

variance of a linear combination of random variable can be computed by the formula below: [(source)](https://en.wikipedia.org/wiki/Variance)
<img src="ext/lincomb_variance.png" style="height:200px">

In [ ]:
def get_portfolio_risk(weights, rat_risks, covariances):
    weight_matrix = np.outer(weights , weights)
    weight_cov_combined = covariances * weight_matrix
    mixed_var = np.sum(np.sum(weight_cov_combined))
    return mixed_var

def get_portfolio_return(weights, rat_risks):
    total_return_rate = np.sum(rat_risks['return_rate'] * weights)
    return total_return_rate

In [ ]:
def optimize_weights(rat_risks, target_return=0.1):
    instruments_count = rat_risks.shape[0]
    init_guess = np.ones(instruments_count) * (1.0 / instruments_count)
    bounds = ((0.0, 1.0),) * instruments_count
    weights = minimize(get_portfolio_risk, init_guess,
                       args=(rat_risks, covariances), method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'eq', 'fun': lambda inputs: 1.0 - np.sum(inputs)},
                                    {'type': 'eq', 'args': (rat_risks,),
                                     'fun': lambda inputs, rat_risks:
                                     target_return - get_portfolio_return(weights=inputs,
                                                                          rat_risks=rat_risks)}
                                   ),
                       bounds=bounds)
    return weights.x, weights.success, weights.status

In [ ]:
weights, success, status = optimize_weights(rat_risks)
print(get_portfolio_risk(results, rat_risks, covariances))
print(success)
print(status)

In [ ]:
weights

In [ ]:
fig, ax = plt.subplots()
ax.scatter(rat_risks['risk'], rat_risks['return_rate'])
for i, stock_name in enumerate(ticker_set):
    ax.annotate(' '.join([stock_name, str(round(weights[i], 5))]), (prediction_dict[stock_name]['valid_sd'], prediction_dict[stock_name]['return_rate']))
plt.title("weigth of each instrument for minimized risk for 0.1 return rate")
plt.xlabel("risk")
plt.ylabel("rate of return")
plt.show()

The weights seem to make sense, with the instruments with higher risk getting a weight of zero and the ones with higher return rate and small risk getting the largest of weights. 
Now let's modify the optimization function to accept and budget constraint as well as output number of shares per instrument (integer) rather than weights.

In [ ]:
def get_portfolio_risk_by_shares(shares, rat_risks, covariances):
    weights = shares / np.sum(shares)
    weight_matrix = np.outer(weights , weights)
    weight_cov_combined = covariances * weight_matrix
    mixed_var = np.sum(np.sum(weight_cov_combined))
    return mixed_var

def get_portfolio_return_by_shares(shares, rat_risks, budget):
    spent = np.sum(rat_risks['price'] * shares)
    unspent = budget - spent
    returns = spent * np.sum(rat_risks['return_rate'] * shares) + unspent
    total_return_rate = (returns/budget) - 1
    return total_return_rate

def budget_constraint(shares, rat_risks, budget):
    prices = np.array(rat_risks['price'])
    unspent = budget - np.sum(shares * prices)
    return(unspent)

In [ ]:
def optimize_shares(rat_risks, target_return=0.1, budget = 2000):
    #normalized_prices = prices / prices.ix[0, :]
    instruments_count = rat_risks.shape[0]
    init_guess = np.ones(instruments_count) * 2 #(1.0 / instruments_count)
    bounds = ((0.0, np.inf),) * instruments_count
    shares = minimize(get_portfolio_risk_by_shares, init_guess,
                       args=(rat_risks, covariances), method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'ineq', 'fun': lambda x: budget_constraint(x, rat_risks, budget)}, #make sure total is less than budget
                                    {'type': 'eq', 'args': (rat_risks, budget), #make the return rate equal to the expected rate
                                     'fun': lambda inputs, rat_risks, budget:
                                     target_return - get_portfolio_return_by_shares(inputs, rat_risks, budget)},
                                    {'type':'eq','fun': lambda x : max([0] + [x[i]-int(x[i]) for i in range(len(x)) if x[i]-int(x[i]) > 0])}, #try to make them as close to integer as possible
                                   ),
                       bounds=bounds
                     )
    return shares.x, shares.success, shares.status, shares.message

In [ ]:
budget = 2000
share_distribution, success, status, message = optimize_shares(rat_risks, budget = budget)
share_distribution
output = rat_risks
output['shares'] = np.floor(share_distribution)

In [ ]:
message

In [ ]:
print(' '.join(['retrun rate', str(get_portfolio_return_by_shares(rat_risks['shares'], rat_risks, budget))]))
print(' '.join(['risk', str(get_portfolio_risk_by_shares(rat_risks['shares'], rat_risks, covariances))]))
print(' '.join(['invested:', str(np.sum(rat_risks['shares'] * rat_risks['price']))]))

In [ ]:
share_distribution

In [ ]:
rat_risks

In [ ]:
#t = pd.DataFrame({'weights':weights})
weights = np.random.randint(1, 10, size = 4)
print(weights)
weight_mat = np.outer(weights , weights)
#pd.pivot_table(t, values = ['weights'], index = 'weights', columns = ['weights'])
#t.info()
print(weight_mat)
#np.stack(weights, 

In [ ]:
weight_mat = np.outer(weights , weights)
mv = covariances.iloc[-4:,-4:] * weight_mat
np.sum(np.sum(mv))
print(mv)

In [ ]:
np.sum(np.sum(mv))

In [ ]:
print(covariances.iloc[-4:,-4:])
print(weight_mat)
print(covariances.iloc[-4:,-4:] * weight_mat)

In [ ]:
-11.118789 * 16

In [ ]:
#funnction copied from matplotlib gallery
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar

In [ ]:
list(d.index.values)[-1]


In [ ]:
covars

In [ ]:
rat_risks.head(10)

Some of the stock data have a lot of missing points over the past 9 years. This might be because they were founded later or had their IPO sometime during this time-period. Below I take a look at the number of stocks with various numbers of missing points and filter the data to the ones with less than 20 NA's. I will impute the missing points by "backward filling".

Alright, now we have the data and for each stock we can take a look at their trend. Change the *ticker_name* below and have a look at the plot. I'm plotting the Microsoft stock prices from some day in December 2012 up to December 2019.

In [ ]:
ticker_name = "MSFT"
import matplotlib.pyplot as plt
plt.plot(d[ticker_name])

In [ ]:
# this function is copied exactly from the deeplearning.ai course on time-series analysis on coursera!
def windowed_dataset(series, window_size, batch_size, shuffle_buffer, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    #dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (tf.expand_dims(window[:-1], axis = -1), window[-1]))
        #dataset = dataset.map(lambda window: (window[0].reshape((len(window[0], 1))), window[1]))
    else:
        dataset = dataset.map(lambda window: (tf.expand_dims(window[:-1], axis = -1), window[-1]))
        #dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
stock_name = "MSFT"
weekly = False

In [ ]:
#d = d.iloc[:, :1]#:int(d.shape[1]/2)]
d = d.loc[:,[stock_name]]
d.describe()

In [ ]:
if weekly:
    d.reset_index(drop = False, inplace = True)
    d['weekday'] = d['Date'].dt.day_name()
    d = d[d['weekday'] == "Friday"]
    d.drop(['Date', 'weekday'], axis = 1, inplace = True)
    print(d.shape)
    print(d.head())
    d.reset_index(drop = True, inplace = True)

In [ ]:
rat_risks.describe()

In [ ]:
p = d.reset_index(drop = True)
plt.plot(p[stock_name])

In [ ]:
window_size = 30
batch_size = 32
shuffle_buffer = 100000
training_points = 350 if weekly else 1500

In [ ]:
#d = d / train_d.max(axis = 0)
train_d = d.iloc[:training_points, :]
valid_d = d.iloc[training_points:, :]
train_df = windowed_dataset(train_d.to_numpy().reshape((len(train_d,))), window_size, batch_size, shuffle_buffer)
normalization_factors = train_d.max()
valid_df = windowed_dataset(valid_d.to_numpy().reshape((len(valid_d,))), window_size, batch_size, shuffle_buffer, shuffle = False)

In [ ]:
train_df
valid_df

In [ ]:
#td = windowed_dataset(np.array(train_d['MSFT']), window_size, batch_size, shuffle_buffer)
#td

In [ ]:
#train_y = train_d[30:]
#train_y.shape
train_d[stock_name].shape
train_d.shape

In [ ]:
train_d.describe()
valid_d.describe()

In [ ]:
def layer_normalize(x, factors = None, denorm = False, input_mean = None, input_std = None):
    if factors is None:
        factors = tf.reduce_max(x, axis = 0, keepdims = False)
        #tf.print(factors)
        tf.print("pre mod shape of x")
        tf.print(tf.shape(x))
        tf.print("shape of factors")
        tf.print(tf.shape(factors))
    input_mean = tf.math.reduce_mean(x, axis = 0)
    input_std = tf.keras.backend.std(x)
    if denorm:
        #x += 1
        factors = 1. / factors
    x /= factors
    '''
    if not denorm:
        if len(x.get_shape()) == 3:
            #tf.print('mean', input_mean)
            #tf.print('std')
            #tf.print('std', input_std)
            #tf.print('oldx')
            #tf.print('shape',x.get_shape())
            #tf.print(x)
            x = (x - input_mean)/input_std
    else:
        #tf.print('mean')       
        #tf.print(len(x.get_shape()))
        x = x * input_std + input_mean
    '''
    #x = (x - tf.keras.backend.mean(x)) / tf.keras.backend.std(x)
    #tf.print("post mod shape of x")
    #tf.print(tf.shape(x))
    return x

In [ ]:
distributional = False
tf.compat.v1.reset_default_graph()
#output_size = train_d.shape[1]
tfd = tfp.distributions
output_dense_size = 2 if distributional else 1

def activations(l, input_mean = None, input_std = None, window_size = None):
    l_0 = (tf.keras.activations.linear(l[...,0])) * input_mean #* input_std) + input_mean #* normalization_factors
    #l_1 = std_multiplier + 
    l_1 = ((tf.keras.activations.linear(tf.abs(l[...,1])))) + 1e-6 # * input_std + input_mean) / window_size) / 0.5
    #tf.print(l_1)
    lnew = tf.stack([l_0, l_1], axis = 1)
    return lnew

def simple_activations(l):
    l_0 = tf.keras.activations.linear(l[...,0])
    l_1 = tf.keras.activations.elu(l[...,1])
    lnew = tf.stack([l_0, l_1], axis = 1)
    return lnew

initializers = "glorot_normal"
activation_name = 'relu'
model = tf.keras.models.Sequential([
  #tf.keras.layers.LayerNormalization(axis = 0),
  #tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'input_mean' : np.mean(train_d)[0], 'input_std': np.std(train_d)[0], 'denorm' : False}, input_shape = (window_size,)),
  #tf.keras.layers.GRU(32, return_sequences = True, kernel_initializer = initializers, activation = activation_name, input_shape = (window_size, 1)), 
  #tf.keras.layers.Conv1D(128, kernel_size = 3),
  #tf.keras.layers.AveragePooling1D(pool_size = 3, padding = 'valid'),
  #tf.keras.layers.LSTM(64, return_sequences=True, kernel_initializer = initializers, activation = activation_name),
  #tf.keras.layers.LSTM(256, return_sequences=True, kernel_initializer = initializers),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(64, return_sequences=True, kernel_initializer = initializers, activation = activation_name),
  #tf.keras.layers.LSTM(128, return_sequences=True, kernel_initializer = initializers),
  #tf.keras.layers.Dropout(0.1),
  #tf.keras.layers.GRU(8, return_sequences=True, kernel_initializer = initializers),
  tf.keras.layers.GRU(64, kernel_initializer = initializers, activation = activation_name, input_shape = (window_size, 1)),
  #tf.keras.layers.LSTM(32, kernel_initializer = initializers, activation = activation_name),
  #tf.keras.layers.Dense(output_dense_size, activation = tf.keras.layers.Activation(lambda x: activations(x, np.mean(train_d)[0], np.std(train_d)[0], window_size)), kernel_initializer = "he_normal"),
  #tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True}),
  #tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=tf.abs(t[..., 0]), scale=0.01*(tf.abs(t[..., 1]))))#-t[...,0]))))#t[...,1])) 
  #                         #scale=(tf.keras.backend.std[...,1])))
  #tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[:,0], scale = t[...,0] + tf.keras.backend.std(t[:,1])))
  #                         #scale=1e-3 + tf.math.softplus(0.05 * t[..., 1:]))),
  #tfp.layers.IndependentNormal(output_size)
  #tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1))
])

if distributional:
    model.add(tf.keras.layers.Dense(
        output_dense_size, activation = 'linear',
        #activation = tf.keras.layers.Activation(lambda x: activations(x, np.mean(train_d)[0], np.std(train_d)[0], window_size)),
        #activation = tf.keras.layers.Activation(lambda x: simple_activations(x)),
        kernel_initializer = "he_uniform"))
    model.add(tfp.layers.DistributionLambda(lambda t: 
                                            tfd.Normal(
                                                loc=tf.abs(t[..., 0]), 
                                                scale= 1e-6 + tf.abs(t[..., 1]) #* normalization_factors
                                            )))
else: 
    #model.add(tf.keras.layers.Dense(128, activation = 'linear'))
    model.add(tf.keras.layers.Dense(1, activation = 'linear', kernel_initializer = initializers))
    #model.add(tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'input_mean' : np.mean(train_d)[0], 'input_std': np.std(train_d)[0], 'denorm' : True}))
#model.add(tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True}))
    
negloglik = lambda y, p_y: -p_y.log_prob(y)
#negcheck = lambda y, yh: tf.math.abs(y - yh[:,0])
#optimizer = tf.keras.optimizers.Adam()#.minimize(cost)
#model.compile(#loss=tf.keras.losses.Huber(),
#              loss=negloglik,
#              #loss = 'mse',
#              #loss = cost,
#              optimizer=optimizer,
#              #optimizer = 'Adam',
#              metrics = ['mae']
#             )
#model.build(input_shape = (batch_size, window_size, None))
#model.summary()
#history = model.fit(df, epochs=2, callbacks = [lr_schedule])
#history = model.fit(train_df, epochs=10, validation_data = valid_df)

loss_function = negloglik if distributional else 'mse' #tf.keras.losses.Huber()
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_function)
model.fit(train_df, epochs=20, verbose=True, validation_data = valid_df)

In [ ]:
xh = model.predict(train_df)
forecasts_x = []
sds = []
#indices = [j for j in range(x.get_shape()[0])]      
#print(indices)
vs = np.array(train_d)
#xs = list(x[i,:,:] for i in indices)
for time in range(len(train_d) - window_size):
    prediction = model.predict(vs[time:time+window_size].reshape((1, window_size, 1)))
    forecasts_x.append(prediction)
    sds.append(prediction - vs[time+window_size])
sd_estimate = np.sqrt(np.sum(np.array(sds)**2) / (training_points - window_size - 1))
forecasts_x = np.array(forecasts_x)[:,0].reshape((training_points - window_size))

In [ ]:
limit_begin = 150
limit_end = 300
plt.plot(np.array(train_d[(window_size + limit_begin): (window_size + limit_end)]), '.')
plt.plot(forecasts_x[limit_begin:limit_end], '.')
plt.plot(forecasts_x[limit_begin:limit_end] + 2*sd_estimate, '-')
plt.plot(forecasts_x[limit_begin:limit_end] - 2*sd_estimate, '-')

In [ ]:
tomorrows_prediction = model.predict(np.array(d[stock_name])[-window_size:].reshape(1, window_size, 1))[0, 0]
tomorrows_prediction

In [ ]:
training_points - window_size

In [ ]:
print(xh[:10])
print(train_d[window_size:window_size + 10])
print((xh - np.array(train_d[window_size:]))[:10])

In [ ]:
forecasts = []
#sds = []
#indices = [j for j in range(x.get_shape()[0])]      
#print(indices)
vs = np.array(valid_d)
#xs = list(x[i,:,:] for i in indices)
for time in range(len(valid_d) - window_size):
    prediction = model.predict(vs[time:time+window_size].reshape((1, window_size, 1)))
    forecasts.append(prediction)
    sds.append(vs[time+window_size] - prediction)
    #forecasts.append(model(vs[time:time+window_size].reshape((1, window_size, 1))))
#print(vs[:window_size])
#print(forecasts)

In [ ]:
print(len(forecasts))
forecasts[1]
if distributional:
    ymeans = np.array([i.mean() for i in forecasts])
    ystdv = np.array([i.stddev() for i in forecasts])
else:
    forecasts = np.array(forecasts[:,0,0])
    sds = np.array(sds)[:,0,0]
forecasts
sd_estimate = np.sqrt(np.sum(sds ** 2)/(sds.shape[0]-1))
sd_estimate

In [ ]:
sds.shape[0]

In [ ]:
forecasts[0].shape
results = np.array(forecasts)[:, 0]
np.max(results)

In [ ]:
plt.plot(np.array(valid_d)[window_size + 100: window_size + 150], '.', label = 'obs')
plt.plot(forecasts[100:150], '.', label = 'pred')
plt.plot(forecasts[100:150] + sd_estimate, '-')
plt.plot(forecasts[100:150] - sd_estimate, '-')
plt.legend()

## draftpad: (nothing interesting below)

In [ ]:
train_df

In [ ]:
x = np.zeros(shape = (batch_size, window_size))
y = np.zeros(shape = (batch_size, 1))
for i, j in valid_df:
    x = i
    y = j
    break

In [ ]:
print(x.shape, y.shape, distributional)

In [ ]:
if (distributional):
    ys = model(x)
    yhat = ys.mean()
    ydev = ys.stddev()
else:
    #ys = pd.DataFrame(model.predict(x))
    ys = model.predict(x)

In [ ]:
ys.shape

In [ ]:
plt.plot(y, '.', label = 'obs')
if distributional:
    plt.plot(yhat, 'r.', label = 'pred')
    plt.plot(yhat + 2*ydev, '-')
    plt.plot(yhat - 2*ydev, '-')
else:
    plt.plot(ys[:50, 0], '.', label = 'pred')
plt.legend()

In [ ]:
ys.shape

In [ ]:
ts = vs[:30].reshape((1, 30))
ts.shape
model.predict(ts)

In [ ]:
vs[time:time+window_size].reshape((window_size))

In [ ]:
plt.plot(y, 'b.')
plt.plot(model(x)[:,0], 'r.')
plt.plot(model(x)[:,0] + model(x)[:,1], '-')

In [ ]:
p = model.predict(valid_df)
#p = p.reshape((1480, 4165))
print(d.shape)
print(p.shape)
print(valid_d.describe())
p = pd.DataFrame(p) #* d.max(axis = 0).reset_index(drop = True)
print(p.describe())

In [ ]:
yhat = model.predict(valid_df)

In [ ]:
mean = yhat.mean()
stddev = yhat.std()
mean_plus_2_stddev = mean - 2. * stddev
mean_minus_2_stddev = mean + 2. * stddev
#plt.plot(mean)#, 'o')
plt.plot(yhat, '.')
plt.plot(vd*normalization_factors + mean_plus_2_stddev, '-')

In [ ]:
vd = valid_d.reset_index(drop = True)
normalization_factors

In [ ]:
def mdn_cost(mu, sigma, y):
    dist = tf.distributions.Normal(loc=mu, scale=sigma)
    return tf.reduce_mean(-dist.log_prob(y))

Since I'm trying to output mean and variance for each stock, I will need two units in my output layer and each will most likely require a different activation function. This can be done [at least more neatly] in funnctional API of keras.

In [ ]:
output_size = train_d.shape[1]
variables = train_d.shape[1]
#output_size = 1
print(output_size)
inputs = tf.keras.Input(shape=(None, variables))
normalization_layer = tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : False})(inputs)
output_1 = tf.keras.layers.LSTM(128, return_sequences=True)(normalization_layer)
output_2 = tf.keras.layers.LSTM(256, return_sequences=True)(output_1)
output_3 = tf.keras.layers.LSTM(256, return_sequences=True)(output_2)
output_4 = tf.keras.layers.LSTM(128, return_sequences=False)(output_3)
#predictions = tf.keras.layers.Dense(1, activation='elu')(output_4)

#sigmoid_out = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
#relu_out = tf.keras.layers.Dense(units=1, activation=tf.nn.relu)
#out = tf.concat([sigmoid_out, relu_out], axis=1)

#denorm_layer = tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})(predictions)

mu = tf.keras.layers.Dense(units=1, activation = 'elu')(output_4)
sigma = tf.keras.layers.Dense(units=1,activation=lambda x: tf.nn.elu(x) + 1)(output_4)

mu_denorm = tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})(mu)
sigma_denorm = tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})(sigma)

out = tf.concat([sigma_denorm, mu_denorm], axis=1)

model = tf.keras.Model(inputs=inputs, outputs=out)

'''
sigmoid_out = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
relu_out = tf.keras.layers.Dense(units=1, activation=tf.nn.relu)
out = tf.concat([sigmoid_out, relu_out], axis=1)
inner_layers.add(out)

denorm_layer = tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})

inner_layers.add(denorm_layer)
model = inner_layers

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-2 * 10**(epoch / 20))
#optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
#optimizer = tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9)
#optimizer = tf.keras.optimizers.SGD(lr=1e-10, momentum = 0.9, clipnorm = 2)
optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.RMSprop()
'''
cost = mdn_cost(mu_denorm, sigma_denorm, y)
optimizer = tf.keras.optimizers.Adam().minimize(cost)
model.compile(#loss=tf.keras.losses.Huber(),
              #loss = 'mse',
              loss = cost,
              optimizer=optimizer,
              #optimizer = 'Adam',
              metrics = ['mae']
             )
#model.build(input_shape = (batch_size, window_size, None))
#model.summary()
#history = model.fit(df, epochs=2, callbacks = [lr_schedule])
history = model.fit(train_df, epochs=10, validation_data = valid_df)#, callbacks = [lr_schedule])

In [ ]:
output_size = train_d.shape[1]
#output_size = 1
print(output_size)
model = tf.keras.models.Sequential([
  #tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]), #if the input is one stock
  tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : False}),
  #tf.keras.layers.Conv1D(128, kernel_size = 3, input_shape = (batch_size, window_size, None)),
  #tf.keras.layers.AveragePooling1D(),
  #tf.keras.layers.Conv1D(256, kernel_size = 3),
  #tf.keras.layers.AveragePooling1D(),
  #tf.keras.layers.Conv1D(256, kernel_size = 1),
  #tf.keras.layers.AveragePooling1D(),
  #tf.keras.layers.Conv1D(256, kernel_size = 1, activation = 'relu'),
  #tf.keras.layers.AveragePooling1D(),
  #tf.keras.layers.GRU(512, return_sequences=True),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.GRU(256, return_sequences=True),
  #tf.keras.layers.GRU(256, return_sequences=True),
  #tf.keras.layers.GRU(256, return_sequences=True),
  #tf.keras.layers.GRU(256),
  #tf.keras.layers.SimpleRNN(128, return_sequences=True),
  #tf.keras.layers.SimpleRNN(256, return_sequences=True),
  #tf.keras.layers.SimpleRNN(512, return_sequences=True),
  #tf.keras.layers.SimpleRNN(256, return_sequences=True),
  #tf.keras.layers.SimpleRNN(128),
  tf.keras.layers.LSTM(128, return_sequences=True),
  tf.keras.layers.LSTM(256, return_sequences=True),
  #tf.keras.layers.Dropout(0.5),
  tf.keras.layers.LSTM(256, return_sequences=True),
  tf.keras.layers.LSTM(128),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.Dropout(0.5),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.LSTM(256, return_sequences=True, activation = 'relu'),
  #tf.keras.layers.LSTM(256, activation = 'tanh', return_sequences=True),
  #tf.keras.layers.LSTM(128),
  #tf.keras.layers.SimpleRNN(64, return_sequences = True),
  #tf.keras.layers.SimpleRNN(128, return_sequences = True),
  #tf.keras.layers.SimpleRNN(256),
  tf.keras.layers.Dense(output_size, activation = 'elu'),
  #tf.keras.layers.Lambda(lambda x: x + 1),
  tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})
])


lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-2 * 10**(epoch / 20))
#optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
#optimizer = tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9)
#optimizer = tf.keras.optimizers.SGD(lr=1e-10, momentum = 0.9, clipnorm = 2)
optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.RMSprop()
model.compile(loss=tf.keras.losses.Huber(),
              #loss = 'mse',
              optimizer=optimizer,
              #optimizer = 'Adam',
              metrics = ['mae']
             )
#model.build(input_shape = (batch_size, window_size, None))
#model.summary()
#history = model.fit(df, epochs=2, callbacks = [lr_schedule])
history = model.fit(train_df, epochs=10, validation_data = valid_df)#, callbacks = [lr_schedule])

In [ ]:
d.to_numpy()[50:55, :8]
train_d.describe()

In [ ]:
#p = model.predict(np.array(d['MSFT']).reshape((len(d['MSFT']), 1)))
p = model.predict(train_df)
#p = p.reshape((1480, 4165))
print(d.shape)
print(p.shape)
p = pd.DataFrame(p)
p2 = p #* d.max(axis = 0).reset_index(drop = True)
p2.describe()

In [ ]:
### MDN MODEL

N_MIXES = 1  # number of mixture components
OUTPUT_DIMS = 1  # number of real-values predicted by each mixture component

model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : False}),
  tf.keras.layers.LSTM(128, return_sequences=True),
  tf.keras.layers.LSTM(256, return_sequences=True),
  tf.keras.layers.LSTM(256, return_sequences=True),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dense(1, activation = 'elu'),
  tf.keras.layers.Lambda(layer_normalize, arguments={'factors': normalization_factors, 'denorm' : True})
])

model.add(mdn.MDN(OUTPUT_DIMS, N_MIXES))
model.compile(loss=mdn.get_mixture_loss_func(1,N_MIXES), optimizer=tf.keras.optimizers.Adam())
#model(train_df)
#model.summary()
history = model.fit(train_df, epochs=10)
p = model.predict(train_df)
y_samples = np.apply_along_axis(mdn.sample_from_output, 1, p, OUTPUT_DIMS, N_MIXES, temp=1.0)

In [ ]:
s = pd.DataFrame({'s' : y_samples[:, 0, 0] * normalization_factors[0]})
s.describe()
#normalization_factors

In [ ]:
p = model.predict(valid_df)
#p = p.reshape((1480, 4165))
print(d.shape)
print(p.shape)
print(valid_d.describe())
p = pd.DataFrame(p) #* d.max(axis = 0).reset_index(drop = True)
print(p.describe())

In [ ]:
print(valid_d.describe())
print(train_d.describe())

In [ ]:
valid_d.tail()

In [ ]:
p2 = model.predict(df)
pd.DataFrame(p2).describe()

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-1, 130000, 170000])

In [ ]:
np.where(d.columns.values == "MSFT")

In [ ]:
p[:, 2572].max()

In [ ]:
t.tickers[0].info

In [ ]:
t = yf.Tickers(' '.join(list(tickers.keys())))

In [ ]:
#df = t.history(period="9y", interval = '1d')
?Tickers.history

In [ ]:
yf.Tickers.history

In [ ]:
t.history(period="9y", interval = '1d', 'Open')

In [ ]:
import mdn

In [ ]:
!pip3 install keras-mdn-layer

In [ ]:
import sys
print(sys.version)

In [ ]:
import mdn

In [ ]:
d

In [ ]:
d.iloc[d.shape[0] - 1][ticker_set]

In [ ]:
ticker_set

In [ ]:
msft = yf.Ticker("MSFT")
temp_hist = msft.history(period="9y", interval="1d")

In [ ]:
ld = d.index.values[-1]
print(ld)
p = pd.to_datetime([ld])
p = p.strftime("%Y-%m-%d")
p.values[0]
#pd.DatetimeIndex(np.datetime_as_string(ld))
last_date = (pd.to_datetime(d.index.values[-1])).strftime("%Y-%m-%d")
print(last_date)

In [ ]:
h = msft.history(start = last_date, interval='1d')


In [ ]:
h.head()

In [ ]:
d_update

In [ ]:
hist = tickers['BKJ']['ticker'].history(start = last_date, interval = "1d")[['Open']]

In [ ]:
hist.drop_duplicates()

In [ ]:
hist

In [ ]:
h2 = hist.drop_duplicates(keep = 'last')

In [ ]:
h2

In [54]:
counter = 0
print(tickers[list(tickers.keys())[counter]]['name'])
print(tickers[list(tickers.keys())[counter]]['business_summary'])

Agilent Technologies, Inc. Common Stock
Agilent Technologies, Inc. provides application focused solutions to the life sciences, diagnostics, and applied chemical markets worldwide. It operates in three segments: Life Sciences and Applied Markets, Diagnostics and Genomics, and Agilent CrossLab. The Life Sciences and Applied Markets segment offers liquid and gas chromatography systems and components; liquid and gas chromatography mass spectrometry systems; inductively coupled plasma mass spectrometry instruments; atomic absorption instruments; microwave plasma-atomic emission spectrometry instruments; inductively coupled plasma optical emission spectrometry instruments; raman spectroscopy; cell analysis plate based assays; flow cytometer; real-time cell analyzer; laboratory software and information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps; and measurement technologies. The Diagnostics and Genomics segment provides reagents, in